In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lifelines as lf
from lifelines.utils.sklearn_adapter import sklearn_adapter


import sklearn as sk
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LinearRegression

In [10]:
# let's get our data
data = pd.read_csv('clean2class/afib_425_2_clean_X2.csv')
data.describe()
T, E = data['duration'], data['event']

In [11]:
data.head()

,duration,event,age,afib_0.0,afib_1.0,sex_-9.0,sex_0.0,sex_1.0,race_0.0,race_1.0
0,203.0,1.0,89.0,0,1,0,0,1,1,0
1,2048.0,1.0,70.0,0,1,0,1,0,0,1
2,1972.0,1.0,97.0,0,1,0,0,1,0,1
3,1074.0,1.0,91.0,0,1,0,1,0,0,1
4,1980.0,1.0,78.0,0,1,0,0,1,0,1


In [12]:
data['event'].value_counts()

1.0    39422
0.0     8856
Name: event, dtype: int64

In [15]:
data['afib_1.0'].value_counts()

1    41435
0     6843
Name: afib_1.0, dtype: int64

In [7]:
X = pd.DataFrame(data.drop('duration', axis=1))
Y = pd.DataFrame(data['duration'])

X['age'] = preprocessing.scale(X['age'])
X

,event,age1,sex_0.0,sex_1.0,race_1.0,race_2.0,race_3.0,race_4.0,race_5.0,race_6.0,race_9.0
0,1.0,1.020130,0,1,0,0,0,0,0,1,0
1,1.0,-0.778459,1,0,1,0,0,0,0,0,0
2,1.0,0.934483,0,1,0,0,0,1,0,0,0
3,1.0,0.934483,1,0,1,0,0,0,0,0,0
4,1.0,-0.264577,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
559243,0.0,0.334953,0,1,0,0,0,0,0,0,1
559244,0.0,0.078012,1,0,0,0,0,0,0,0,1
559245,0.0,-0.778459,1,0,0,0,0,0,0,0,1
559246,0.0,-0.007635,1,0,0,0,0,0,0,0,1


In [6]:
trainX, valX, trainY, valY = model_selection.train_test_split(X, Y)
trainX, valX, trainY, valY = pd.DataFrame(trainX), pd.DataFrame(valX), pd.DataFrame(trainY), pd.DataFrame(valY)

trainX.columns=['event','age','s1','s2','r1','r2','r3','r4','r5','r6','r9'] 
valX.columns=['event','age','s1','s2','r1','r2','r3','r4','r5','r6','r9'] 

### Cox Regression

In [7]:
CoxRegression = sklearn_adapter(lf.CoxPHFitter, event_col='event')

sk_cph = CoxRegression(penalizer=.05)
sk_cph.fit(trainX, trainY)
sk_cph.lifelines_model.print_summary()

In [8]:
sk_cph.score(valX, valY)

0.6101978826432392

### Linear Regression

In [9]:
lrg = LinearRegression()
lrg.fit(trainX, trainY)
print(lf.utils.concordance_index(valY, lrg.predict(valX), event_observed=valX['event']))

0.6137510799236435


### Weibull AFT

In [10]:
WeibullAFT = sklearn_adapter(lf.WeibullAFTFitter, event_col='event')

sk_aft = WeibullAFT()
sk_aft.fit(trainX, trainY)
sk_aft.lifelines_model.print_summary()

C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\lifelines\fitters\__init__.py:1953: StatisticalWarning: The diagonal of the variance_matrix_ has negative values. This could be a problem with WeibullAFTFitter's fit to the data.

It's advisable to not trust the variances reported, and to be suspicious of the fitted parameters too.

  warnings.warn(warning_text, utils.StatisticalWarning)


In [11]:
sk_aft.score(valX, valY)

0.6101810328561372

### Aalen's Additive Regression

In [12]:
aff = lf.AalenAdditiveFitter(coef_penalizer=.05)
aff.fit(data, duration_col='duration', event_col='event')
aff.print_summary()

### K-Fold Cross-Validation

##### Aalen's

In [13]:
print(np.mean(lf.utils.k_fold_cross_validation(lf.AalenAdditiveFitter(coef_penalizer=.1), data, duration_col='duration', event_col='event', scoring_method='concordance_index')))

0.5


##### Weibull AFT

In [14]:
print(np.mean(lf.utils.k_fold_cross_validation(lf.WeibullAFTFitter(), data, duration_col='duration', event_col='event', scoring_method='concordance_index')))

C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\lifelines\fitters\__init__.py:1953: StatisticalWarning: The diagonal of the variance_matrix_ has negative values. This could be a problem with WeibullAFTFitter's fit to the data.

It's advisable to not trust the variances reported, and to be suspicious of the fitted parameters too.

  warnings.warn(warning_text, utils.StatisticalWarning)
C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\lifelines\fitters\__init__.py:1953: StatisticalWarning: The diagonal of the variance_matrix_ has negative values. This could be a problem with WeibullAFTFitter's fit to the data.

It's advisable to not trust the variances reported, and to be suspicious of the fitted parameters too.

  warnings.warn(warning_text, utils.StatisticalWarning)
C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\lifelines\fitters\__init__.py:1953: StatisticalWarning: The diagonal of the variance_matrix_ has negative v

0.6241597647776217


##### Cox

In [15]:
print(np.mean(lf.utils.k_fold_cross_validation(lf.CoxPHFitter(penalizer=.05), data, duration_col='duration', event_col='event', scoring_method='concordance_index')))

0.6243309518474884


### scikit-survival models

In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM

sns.set_style("whitegrid")

In [17]:
data['event'] = data['event'].astype(bool)
X = pd.DataFrame(data.drop(['duration','event'], axis=1))
Y = pd.DataFrame(data[['event','duration']])

X = pd.DataFrame(preprocessing.scale(X))

trainX, valX, trainY, valY = model_selection.train_test_split(X, Y)
trainX, valX, trainY, valY = pd.DataFrame(trainX), pd.DataFrame(valX), pd.DataFrame(trainY), pd.DataFrame(valY)

trainY = trainY.to_records(index=False)

valY = valY.to_records(index=False)

trainX.columns=['age','s1','s2','r1','r2','r3','r4','r5','r6','r9']
valX.columns=['age','s1','s2','r1','r2','r3','r4','r5','r6','r9'] 

##### Cox variation with elastic net penalty

In [18]:
from sksurv.linear_model import CoxnetSurvivalAnalysis

cphnet = CoxnetSurvivalAnalysis()
cphnet.fit(trainX, trainY)

C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\sksurv\linear_model\coxnet.py:211: FutureWarning: The default value of alpha_min_ratio will depend on the sample size relative to the number of features in 0.13. If n_samples > n_features, the current default value 0.0001 will be used. If n_samples < n_features, 0.01 will be used instead.
  FutureWarning)


CoxnetSurvivalAnalysis(alpha_min_ratio=0.0001, alphas=None, copy_X=True,
                       fit_baseline_model=False, l1_ratio=0.5, max_iter=100000,
                       n_alphas=100, normalize=False, penalty_factor=None,
                       tol=1e-07, verbose=False)

In [19]:
cphnet.score(valX, valY)

0.6286205386771473

##### sksurv's Cox implementation

In [20]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

cph = CoxPHSurvivalAnalysis()
pd.Series(cph.fit(trainX, trainY))

C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\sksurv\linear_model\coxph.py:421: LinAlgWarning: Ill-conditioned matrix (rcond=1.13869e-23): result may not be accurate.
  overwrite_a=False, overwrite_b=False, check_finite=False)
C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\sksurv\linear_model\coxph.py:421: LinAlgWarning: Ill-conditioned matrix (rcond=2.37314e-17): result may not be accurate.
  overwrite_a=False, overwrite_b=False, check_finite=False)
C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\sksurv\linear_model\coxph.py:421: LinAlgWarning: Ill-conditioned matrix (rcond=6.76082e-21): result may not be accurate.
  overwrite_a=False, overwrite_b=False, check_finite=False)


0    CoxPHSurvivalAnalysis(alpha=0, n_iter=100, tie...
dtype: object

In [21]:
cphnet.score(valX, valY)

0.6286205386771473